# Network Modifications - Roadways

In this notebook a copy of our baseline NetworkDataset is made to be modified reflecting the candidate improvement

In [1]:
import arcpy
import os
from arcgis.features import SpatialDataFrame
import pandas as pd

arcpy.CheckOutExtension("network")

# set base path - jupyter or arcgis
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    print("base path must be explicitly set when running in arcgis pro")
    base_path = r"c:\wfrc\ato" # update this
except OSError:
    base_path = "."

base_gdb = os.path.join(base_path, "nd.gdb")

target_gdb =  os.path.join(base_path, "mod_drive.gdb")
if not os.path.isdir(target_gdb):
    arcpy.management.CreateFileGDB(base_path, "mod_drive")

arcpy.env.workspace = target_gdb

base path must be explicitly set when running in arcgis pro


In [2]:
# Copy our baseline network dataset to our dataset for modification
arcpy.management.Copy(
    os.path.join(base_gdb, r"NetworkDataset"), 
    os.path.join(target_gdb, "NetworkDataset")
)

<Result 'c:\\wfrc\\ato\\mod_drive.gdb\\NetworkDataset'>

In [3]:
# delete existing network
arcpy.management.Delete(os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND"))

<Result 'true'>

# Make Edits

1. Add BikePedAuto layer to current map
2. Follow the instructions below for the appropriate section to make edits


Project Type | Action 
---- | ----
New construction (line) | Add new line to network and make appropriate connections 
Widening (line) | Reduce travel time along links to free flow speed (Julie will ask Suzy what the travel time benefit is in the model) 
Operational (line) | Reduce travel time along links to average of free flow speed and peak hour speed 
Restripe (when is basically widening) (line) | Reduce travel time along links to free flow speed 
New interchange (point)  | Add connections in network 
Grade-separated crossing | Add new line across intersection (no intersection impedance) 



## New construction (line)

Note, when editing lines, take care to use snapping--overlapping verticies need to precisely connect.

connecting the end point of a new line to the mid-point of an existint segment will not work. The existing segment must be split at the connection point.

1. Add new line to network (or copy existing line from source layer). Edit --> Copy --> 
2. Edit vertices and ensure that beginning / end points connect to other beginning / end points. 
3. Select the intersecting road segments and use the Planarize Tool to split the intersecting lines (requires ArcGIS Pro Standard or Advanced)
5. Update attributes of new feature (hint, use the Transfer Attributes tool to copy attributes from a similar nearby segment)
6. Select all affected features and update length attribute

ped = 3 mph
bike = 11 mph

* Oneway = B
* Speed (choose an appropriate speed based on functional class, etc.)
* AutoNetwork = Y (or N for ped/transit only) -- actually new transit service shouldn't be added to the BikePedAuto but to Transit Routes
* BikeNetwork = Y
* Ped Network = Y
* DriveTime = calculate based on length (note: length is in meters) and speed
* hierarchy: 1 = Interstate 2 = Major Arterial 3 = Local

In [9]:
# UPDATE LENGTHS FOR SELECTED FEATURES - SELECT ONLY AFFECTED FEATURES TO KEEP RUN TIME REASONABLE
# this will likely throw some TypeErrors if the selectio includes non-roadway segments - ignore these!
if int(arcpy.management.GetCount("BikePedAuto")[0]) < 100:
    arcpy.management.CalculateField("BikePedAuto", "Length_Miles", '!shape.length@miles!', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "DriveTime", '!Length_Miles! / (!Speed! / 60)', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "PedestrianTime", '!Length_Miles! / (3 / 60)', "PYTHON3", None, "DOUBLE")
    arcpy.management.CalculateField("BikePedAuto", "BikeTime", '!Length_Miles! / (11 / 60)', "PYTHON3", None, "DOUBLE")
else:
    print("Warning: operation will affect more than 100 features - did you select only the intended target?")

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'


In [3]:
# don't forget to clear the selection before creating the new network d"ataset
arcpy.management.SelectLayerByAttribute("BikePedAuto", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


# save edits!

In [16]:
# create network dataset from template
arcpy.na.CreateNetworkDatasetFromTemplate(
    os.path.join(base_path, "ND_Template_Aug21.xml"),                           
    os.path.join(target_gdb, "NetworkDataset")
)

<Result 'c:\\wfrc\\ato\\mod_drive.gdb\\NetworkDataset\\NetworkDataset_ND'>

In [17]:
# finally, build the dataset
arcpy.na.BuildNetwork(os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND"))

<Result 'c:\\wfrc\\ato\\mod_drive.gdb\\NetworkDataset\\NetworkDataset_ND'>

## Widening (line)

## Operational (line)

## Restripe (when is basically widening) (line) 
Reduce travel time along links to free flow speed 


## New interchange (point) 
Add connections in network 
Grade-separated crossing | Add new line across intersection (no intersection impedance) 